# COGS 108 - Project Proposal

# Names

- David An 
- Delvin Bajoua 
- Eric Chun 
- Joshua Huang
- Zeyad Almoaigel

# Research Question

How do the occurrences of various types of traffic accidents such as hit and runs, DUIs, and speeding vary among different vehicle makes and models in San Diego? 

Sub-questions:

 - How does the frequency and severity of traffic collisions vary by vehicle type over the specified time period?

 - Are certain violation types more strongly associated with severe injuries or fatalities compared to others?

 - What is the relationship between hit-and-run incidents and the severity of injuries or fatalities in traffic collisions?

 - How has the prevalence of specific vehicle makes and models changed over time, and what impact, if any, has this had on collision severity?

 - Are there geographic or demographic trends that correlate with higher rates of severe injuries or fatalities in traffic collisions?

## Background and Prior Work

With an increase in traffic congestion and drivers on the road, San Diego is seeing a spike in road incidents. There is a growing concern, especially considering how the district attorney's office has prosecuted 225 fatal cases for driving under the influence (DUIs) over the past 10 years since 2023. <a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1) Many institutions are looking into the issue, with UC San Diego's Herbert Wertheim School of Public Health and Human Longevity Science leading a program to make roads safer for everyone, including drivers, pedestrians, and cyclists. This effort is supported by a $360,000 grant from the California Office of Traffic Safety through the NHTSA. <a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2) With the increase in accidents comes the question how this varies among different types of vehicles, makes, and models. According to the Times of San Diego, distracted driving, speeding, and DUIs are cited as the primary causes of accidents in the region.  <a name="cite_ref-3"></a>[<sup>3</sup>](#cite_note-3) Exploring these distributions could unveil specific focus groups or brands associated with accidents, providing invaluable insights into real-world traffic occurrences and vehicular accidents. An analysis of car accident data from 2017 to 2022 also sheds light on the variations in accident occurrences. <a name="cite_ref-4"></a>[<sup>4</sup>](#cite_note-4) Specific types of violations, such as failure to yield or unsafe turns may additionally demonstrate distinct patterns that may be influenced by vehicle characteristics and driving behaviors. Understanding these nuances is essential for informing policies and strategies that effectively address the occurrence of specific types of accidents and promote road safety for all road users.

### Sources

1. <a name="cite_note-1"></a> [^](#cite_ref-1) Fleming, Omari. (5 Dec 2023) Deadly DUI numbers in San Diego County nearing all-time high: District Attorney. *NBC San Diego*. https://www.nbcsandiego.com/news/local/deadly-dui-numbers-in-san-diego-county-nearing-all-time-high-district-attorney/3373245/ 

2. <a name="cite_note-2"></a> [^](#cite_ref-2) Delong, Tyler. (9 November 2023) Grant Helps Program Expand Distracted Driving Education to Online Learning. *UC San Diego Today*. https://today.ucsd.edu/story/grant-helps-program-expand-distracted-driving-education-to-online-learning 

3. <a name ="cite_note-3"> </a> [^](#cite_ref-3) (4 March 2024) Understanding the Leading Causes of Car Accidents in San Diego. *Times of San Diego*. https://timesofsandiego.com/sponsored/2024/03/04/understanding-the-leading-causes-of-car-accidents-in-san-diego/ 

4. <a name ="cite_note-4"> </a> [^](#cite_ref-4) City of San Diego Number of Car Accidents by Car Accident Type from 2017 to 2022. *Hamparyan Law*. https://www.hamparyan.com/city-san-diego-number-car-accidents-car-accident-type-2017-2022/ 



# Hypothesis


We believe that car brands favored by younger generations or falling under the higher luxury/sport categories are more likely to be involved in collision accidents. In terms of reasons for these accidents, our observations suggest that speeding, driving under the influence, and making unsafe turns are the most prevalent factors for incidents in San Diego overall as well as for vehicles classified in the luxury/sport category.

# Data

1. The ideal dataset
    - The ideal dataset should include the cause of a traffic collision, vehicle makes, and vehicle models in San Diego so that we can answer our primary research question. Then, to answer our sub-questions, it would be nice if there is an indication of the severity of a collision that we can utilize such as the number of people who got injured or the amount of damage on vehicles. Additionally, to do geospatial analysis, it would be nice to have where each collision occurred. To examine frequency correctly, it would be ideal if there is a government organization that keeps all the traffic collision records that we can use. If this is the case, it's likely to be that the data is recorded by different police officers across different regions of San Diego. Some might use different abbreviations for collision causes and some might exclude some information such as specifying the exact model type. So we shouldn't expect 100% accuracy on the recorded data and explore this area during our EDA.

2. Potential real datasets
    - Traffic collisions data based in San Diego from DataSD (https://data.sandiego.gov/datasets/police-collisions-details/)

## Dataset 1:

In [9]:
import pandas as pd

original_df = pd.read_csv('pd_collisions_details_datasd.csv', low_memory=False)
beat = pd.read_csv('https://seshat.datasd.org/gis_police_beats/pd_beat_codes_list_datasd.csv').set_index('beat')
beat_dict = beat.to_dict()['neighborhood']
collisions_sd = original_df.copy()
collisions_sd['date_time'] = pd.to_datetime(collisions_sd['date_time'])
collisions_sd['year'] = collisions_sd['date_time'].dt.year
collisions_sd['district'] = collisions_sd['police_beat'].apply(lambda x: beat_dict[x] if x in beat_dict else 'unknown')
collisions_sd['charge_desc'] = collisions_sd['charge_desc'].str.lower()

# Address
collisions_sd['street_address'] = (collisions_sd['address_no_primary'].astype(str)
                                   + collisions_sd['address_pd_primary']
                                   .apply(lambda x: x if x == ' ' else " " + x + " ")
                                     + collisions_sd['address_road_primary'] + " "
                                  + collisions_sd['address_sfx_primary'])
collisions_sd['street_address'] = collisions_sd['street_address'].str.strip().str.title().fillna("Unknown")
collisions_sd = collisions_sd.drop(['address_no_primary', 'address_pd_primary', 
                                    'address_road_primary', 'address_sfx_primary', 'police_beat'], axis = 1)
collisions_sd['at_intersection'] = ((collisions_sd['address_pd_intersecting'] != ' ') 
                                    | (collisions_sd['address_name_intersecting'] != ' ') 
                                   | (collisions_sd['address_sfx_intersecting'] != ' '))
collisions_sd['street_address_intersection'] = (collisions_sd['address_pd_intersecting']
                                   + collisions_sd['address_name_intersecting'].astype(str)
                                   .apply(lambda x: x if x == ' ' else " " + x + " ")
                                     + collisions_sd['address_sfx_intersecting'])
collisions_sd['street_address_intersection'] = collisions_sd['street_address_intersection'].apply(
    lambda x: x if x != "   " else "None").fillna("Unknown")
collisions_sd['hit_run_lvl'] = collisions_sd['hit_run_lvl'].fillna("Not applicable").str.title()

collisions_sd = collisions_sd[
    (collisions_sd['person_role']=='DRIVER') &  # Filtering to only include DRIVER since we are interested in collision causes by driver
    (collisions_sd['year']>=2017)  # Filtering >= 2017 due to few observations in 2015 and 2016
]

collisions_sd.drop(['person_veh_type',
                    'person_role',
                    'address_pd_intersecting',
                    'address_name_intersecting',
                    'address_sfx_intersecting'], axis=1, inplace=True)

collisions_sd.head(3)

,report_id,date_time,person_injury_lvl,veh_type,veh_make,veh_model,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl,year,district,street_address,at_intersection,street_address_intersection
5843,180393,2017-01-05 08:30:00,NaN,AUTOMOBILE,ACURA,MDX,MISC-HAZ,VC,miscellaneous hazardous violations of the vehi...,0,0,Not Applicable,2017,Sorrento Valley,3800 Sorrento Valley Boulevard,False,None
5844,180418,2017-01-05 10:46:00,NaN,AUTOMOBILE,VOLVO,V70,22106,VC,starting parked vehicles or backing,1,0,Not Applicable,2017,North Clairemont,4800 Clairemont Drive,False,None
5845,175065,2017-01-06 13:45:00,NaN,NaN,NISSAN,SENTRA,21703,VC,following too closely,0,0,Misdemeanor,2017,Oak Park,2100 54Th Street,False,None


The crux of the data here covers all vehicle accidents in San Diego from 2015 to present. This data set is grouped to cover all types of brands and the collisions accidents they got into a singular dataframe. The most important sections will be brand and accident type (found under 'charge_desc') which will be the crux behind the true exploration of the data. Some other potentially useful columns we will use include the year and the location for exploration on aggregation on potential differences in the distribution based on the results we get from our main question. We also have an injured and killed count for further exploration into the severity of accidents. Note this data was already cleaned based on certain choices made to exclude, including person role in accident and the specific violation section. These exclusions were mainly chosen due to believed irrelevance to our current goals. One particular column of interest is the 'person_veh_type' column removed. Some of this information is useful (it points out potential SUVs and jeeps for vehicle shape types for example) but the information is currently too messy and still contains large amounts of irrelevant/less helpful data labels along with missing values. Thus it is uncertain how useful this column will be to our study even with extensive cleaning.

# Ethics & Privacy

 - All the data was largely available online for free and avoided specific user information (license plate, ids, etc.)
   - If personal data is included, such as addresses, we will remove it from the data set as it is irrelevant.
 - It's hard to say exactly if the data is focused on a particularly population (for example race) but based on the sources provided, that does not seem to be the case
 - We'll mainly need to pay attention to the data sources to examine specific biases should they exist, mingling between one another should keep the data fair and not narrow minded towards one aspect. We should also keep update with trends and brand research that may come from unintended biases (for example Teslas might only belong to a certain group that may get label incorrectly due to data)
 - Mainly the idea of brand labeling as negative unintentionally when we're just trying to explore this potential relationship to help clarify whether its relevant or even exists.
 - Bias issues will largely be labeled by disclaimers. If we can avoid it, we will but if it's too integrated with the data, the next step is pinpoint and point out this fact to an audience and clarify it to avoid potential bias forming because of it
 - Whatever conclusion we reach that is supported by the data needs to be as unbias as possible, i.e. no negative connotations to a specific group of drivers like Tesla drivers etc.

# Team Expectations 

* *Main general focus of communication will be through discord, responses will be at maximum 24 hours with a reply (or just an emote) Main exception will be Zeyad who may be less handy with discord and will email more often should need to*
* *Weekly meetings every Saturday, usually around 6 pm to discuss*
* *General Work expectations of contributions, at the very least looking into research/checking out what work has been done and making suggestions*
* *Check in on how every's one feeling or if they're busy, communicate how they're feeling in terms of work to know how every is managing and fill in gaps if need be*
* *If something out of the blue pops up, message and discuss properly as soon as possible*
* *If any issues or questions in general, again simply message and ask*

# Project Timeline Proposal

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 4/13  |  6 PM | Brainstorm possible topics for our project  | Determine best form of communication; Discuss and decide on final project topic; Discuss which type of question we want to ask; begin background research | 
| 4/20  |  6 PM |  Search which dataset we can use | Discuss ideal dataset(s) and ethics; Discuss research question we want to answer; Find which dataset we can use | 
| 4/27  | 5:30 PM  | Briefly look over the datasets; Draft project proposal  | Edit, finalize, and submit proposal |
| 5/4  | 6 PM  | Data exploration and cleaning | Further discussion of what's usable and plan out report  |
| 5/11  | 6 PM  | Data plotting and graph exploration | Discuss on process and check out how we're doing in terms of conclusion |
| 5/18  | 6 PM  | Finalize and submit Data checkpoint (5/17) | Discuss results finding and plan out story telling |
| 5/25  | 6 PM  | work over time on results and conclusions | Explore current format of project see if everything look in reasonable shape |
| 6/1  | 6 PM  | Finalize and submit EDA (5/31) | Discus any major final last changes |
| 6/8  | 6 PM  | Finalize final project; Edit final video | Submit Final Project & Video & Team eval survey (6/12) |